# Trabajo Práctico N.° 3

## Imports y configs

In [ ]:
# un poco menos de warnings de tensorflow
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# de python, para especificar rutas de archivos y directorios
from pathlib import Path

# lib para trabajar con arrays
import numpy as np
import pandas as pd

# lib que usamos para mostrar las imágenes
import matplotlib.pyplot as plt

# libs que usamos para construir y entrenar redes neuronales, y que además tiene utilidades para leer sets de 
# imágenes
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input, Dropout, Convolution2D, MaxPooling2D, Flatten
from tensorflow.keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.applications import VGG16
from tensorflow import keras
from tensorflow.keras import layers
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten

# libs que usamos para tareas generales de machine learning. En este caso, métricas
from sklearn.metrics import accuracy_score, confusion_matrix

# configuración para que las imágenes se vean dentro del notebook
%matplotlib inline

import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# Análisis exploratorio sobre el conjunto de datos

Para que esto funcione, se debe bajar y descomprimir el archivo del siguiente [enlace](https://www.kaggle.com/jessicali9530/celeba-dataset). El mismo contiene:
* **img_align_celeba**: imágenes de rostros, recortadas y alineadas.
* **list_eval_partition.csv**: partición recomendada para las imagenes. Relaciona cada imagen con el atributo *partition*:
    * "0" si pertenece al conjunto de entrenamiento.
    * "1" si pertenece al conjunto de validación.
    * "2" si pertenece al conjunto de prueba.
* **list_bbox_celeba.csv**: información del cuadro delimitador para cada imagen.
* **list_landmarks_align_celeba.csv**: puntos de referencia (ojo izquierdo, ojo derecho, nariz, boca izquierda, boca derecha) y sus respectivas coordenadas.
* **list_attr_celeba.csv**: etiquetas (40 en total) de atributos para cada imagen. "1" representa positivo mientras que "-1" representa negativo.

En nuestro caso, utilizaremos las imágenes, *list_eval_partition.csv* y *list_attr_celeba.csv* para desarrollar un modelo que sea capaz de detectar si una persona tiene o no barba a partir de una foto. Entonces, nuestro target será el atributo *No_Beard*, que determina si una persona en una imagen no tiene barba.

In [ ]:
# Modificar por la ruta donde se encuentran los archivos
# current_path = '/home/matias/Documentos/archive/'
current_path = '/home/yair/Files/archive/'

imgs_path = Path(current_path, 'img_align_celeba/img_align_celeba')
attr_path = Path(current_path, 'list_attr_celeba.csv')
partitions_path = Path(current_path, 'list_eval_partition.csv')

# Importamos y unimos los datasets de atributos y particiones
df_attr = pd.read_csv(attr_path, usecols=['image_id','No_Beard'])
df_partitions = pd.read_csv(partitions_path)
df = df_attr.merge(df_partitions, on="image_id", how="left")

df.sample(5)

In [ ]:
# Restablecemos los valores de la columna No_Beard
df.loc[df['No_Beard'] == -1,'No_Beard'] = "Barba"
df.loc[df['No_Beard'] == 1,'No_Beard'] = "No_Barba"

# Dividimos el dataframe en función de la columna partition
train = df.loc[df['partition'] == 0]
validation = df.loc[df['partition'] == 1]
test = df.loc[df['partition'] == 2]

# Eliminamos la columna partition de los 3 conjuntos
train = train.drop(['partition'],axis=1)
validation = validation.drop(['partition'],axis=1)
test = test.drop(['partition'],axis=1)

train.sample(5)

In [ ]:
print("Cantidad total de imágenes:", len(df.image_id))
print("Cantidad de imágenes en train:", len(train.image_id))
print("Cantidad de imágenes en validation:", len(validation.image_id))
print("Cantidad de imágenes en test:", len(test.image_id))

## Volumetría de los datos

In [ ]:
from PIL import Image

img_path = Path(str(imgs_path) + '/000002.jpg') # Este solo es un ejemplo
img = Image.open(img_path)

width, height = img.size
img_array = np.array(img)
depth = img_array.shape[2]

print(f'Alto de la imagen: {height} píxeles')
print(f'Ancho de la imagen: {width} píxeles')
print(f'Profundidad de la imagen (RGB): {depth}')

## Distribución de la variable a predecir

In [ ]:
category_count = df["No_Beard"].value_counts()
print(f'Cantidad total de imágenes de personas sin barba: {category_count.values[0]}')
print(f'Cantidad total de imágenes de personas con barba: {category_count.values[1]}')

In [ ]:
fig, axs = plt.subplots(nrows=1, ncols=4, figsize=(16, 16))

df.No_Beard.value_counts().plot.pie(autopct='%1.0f%%', label='', ax=axs[0])
axs[0].set_title('Dataset completo')

train.No_Beard.value_counts().plot.pie(autopct='%1.0f%%', label='', ax=axs[1])
axs[1].set_title('Train')

validation.No_Beard.value_counts().plot.pie(autopct='%1.0f%%', label='', ax=axs[2])
axs[2].set_title('Validation')

test.No_Beard.value_counts().plot.pie(autopct='%1.0f%%', label='', ax=axs[3])
axs[3].set_title('Test')

Podemos concluir entonces que la métrica F1 es la que mejor se ajusta al problema ya que se tiene un conjunto de datos que se encuentra desbalanceado. Esto nos permite ponderar el rendimiento del modelo con respecto a los falsos positivos y falsos negativos.

## Estructura y tipo de las imágenes

Por cuestiones de tiempo de entrenamiento, vamos a aplicar cambios a las imágenes al armar los conjuntos de entrenamiento. Se va a reducir la dimensionalidad (alto y ancho), pasando de 218 x 178 píxeles a 64 x 64 píxeles.

In [ ]:
# Rescalaremos los valores de las imágenes
datagen = ImageDataGenerator(
    rescale=1./255,
)
# Para crear datasets de imágenes para Keras
IMAGE_SIZE = (64, 64)
BATCH_SIZE = 100
parameters = dict(
    x_col='image_id',
    y_col='No_Beard',
    target_size=(IMAGE_SIZE),
    class_mode='categorical',
    batch_size=BATCH_SIZE,
)

# Generar datasets
train_generator = datagen.flow_from_dataframe(train, str(imgs_path) + "/",**parameters)
validation_generator = datagen.flow_from_dataframe(validation,str(imgs_path) + "/",**parameters)
test_generator = datagen.flow_from_dataframe(test, str(imgs_path) + "/",**parameters)

In [ ]:
# Obtener un bloque (imagenes y etiquetas)
train_imgs, train_labels = train_generator.next()

In [ ]:
# Estructura de una imagen
train_imgs[0]

In [ ]:
# Dibujar imagenes de ejemplo
def draw_images(dataset):
    cant_imgs = 15
    row_plt = 3
    col_plt = 5
    images, labels = dataset.next()
    for i in range(cant_imgs):
        ax = plt.subplot(row_plt, col_plt, i + 1)
        plt.imshow(images[i])
        plt.title(labels[i])
        plt.axis("off")

In [ ]:
draw_images(train_generator)

En los ejemplos anteriores podemos visualizar imagenes de tipo:
* [0. 1.] indica que la persona no tiene barba.
* [1. 0.] indica que la persona tiene barba.

# Machine Learning

## Entrenamiento de modelos

In [ ]:
base_vgg_model = tf.keras.applications.vgg19.VGG19(weights='imagenet', include_top=False, input_shape= IMAGE_SIZE + (3,))

In [ ]:
base_vgg_model.trainable = False
base_vgg_model.summary()


In [ ]:
vgg_model = Sequential(
    [
    base_vgg_model,
    Flatten(),
    Dense(256,activation='relu'),
    Dense(256,activation='relu'),
    Dense(64,activation='relu'),
    Dense(32, activation='relu'),
    Dense(32, activation='relu'),
    Dense(2, activation='sigmoid')
]
)

In [ ]:
vgg_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
earlystop = EarlyStopping(patience=10)
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                            patience=4, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.001)
callbacks = [earlystop, learning_rate_reduction]

history = vgg_model.fit(train_generator, validation_data = validation_generator
                        , validation_steps=len(validation_df)//BATCH_SIZE
                        ,steps_per_epoch=len(train_df)//BATCH_SIZE,
                        epochs=10, verbose = 1, callbacks=callbacks)



In [ ]:
""""""
pretrained_model = VGG16(include_top=False, input_shape= IMAGE_SIZE + (3,))
pretrained_model.trainable = False

vgg16_model = Sequential(
    [
    pretrained_model,
    #Convolution2D(filters=8, kernel_size=(2, 2), strides=1, activation='tanh'),
    #MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(50,activation='relu'),
    Dense(32, activation='relu'),
    Dense(2, activation='softmax')
]
)

vgg16_model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy',],
)

In [ ]:
model_weights_at_epochs = {}

class OurCustomCallback(Callback):
    def on_epoch_end(self, epoch, logs=None):
        model_weights_at_epochs[epoch] = self.model.get_weights()
        
history = vgg16_model.fit(
    train_generator, validation_data = validation_generator
                        , validation_steps=len(validation)//BATCH_SIZE
                        ,steps_per_epoch=len(train)//BATCH_SIZE,
                        epochs=10, verbose = 1, callbacks=[OurCustomCallback()]
)

In [ ]:
from tensorflow.keras.datasets import fashion_mnist
train, test = fashion_mnist.load_data()
(x_train, y_train) = train 
(x_test, y_test) = test

In [ ]:
train, test = fashion_mnist.load_data()
(x_train, y_train) = train 
(x_test, y_test) = test

In [ ]:
y_train[1]

In [ ]:
x_train[1]

In [ ]:
train_generator.head()

In [ ]:
train_generator = validation_datagen.flow_from_dataframe(
    train, 
    '/home/matias/Documentos/archive/img_align_celeba/img_align_celeba'+'/', 
    x_col='image_id',
    y_col='No_Beard',
    target_size=IMAGE_SIZE,
    class_mode='binary',
    batch_size=BATCH_SIZE,
    #validate_filenames=False
)

data_array, labels_array = train_generator.next()
    
data_array = []
labels_array = []

for _ in range(BATCH_SIZE):
    batch_data, batch_labels = train_generator.next()
    data_array.append(batch_data)
    labels_array.append(batch_labels)

data_array = np.concatenate(data_array)
labels_array = np.concatenate(labels_array)


In [ ]:
labels_array[0]

In [ ]:
# función para mostrar imágenes
def mostrar_imagenes(entradas, salidas):
    plt.figure(figsize=(10,10))
    for i in range(10):
        plt.subplot(6,6,i+1)
        plt.xticks([])
        plt.yticks([])
        plt.grid(False)
        plt.imshow(entradas[i])
        plt.xlabel(salidas[i])
    plt.show()

mostrar_imagenes(data_array, labels_array)


In [ ]:
data_array[0]

In [ ]:
train_generator[0][0]